In [1]:
print("starting")
import sys
from ipywidgets.widgets.widget_output import Output
sys.path.append("/scr3/jruffio/shubh/breads/")
import numpy as np
import matplotlib.pyplot as plt
from  scipy.interpolate import interp1d
import os
import astropy.io.fits as pyfits
from pathlib import Path
from breads.fit import fitfm
from copy import deepcopy
import multiprocessing as mp
from itertools import repeat
from multiprocessing.pool import ThreadPool
from concurrent.futures import ProcessPoolExecutor as Pool

numthreads = 16

print("Importing mkl")
try:
    import mkl
    mkl.set_num_threads(1)
except:
    pass

print("Importing breads")
from breads.instruments.OSIRIS import OSIRIS
from breads.search_planet import search_planet
from breads.fm.hc_splinefm import hc_splinefm
from breads.fm.hc_no_splinefm import hc_no_splinefm
from breads.fm.hc_hpffm import hc_hpffm
from breads.injection import inject_planet, read_planet_info
import arguments

star = "SR3"
dir_name = arguments.dir_name[star]
tr_dir = arguments.tr_dir[star]
sky_calib_file = arguments.sky_calib_file[star]
files = os.listdir(dir_name)

starting
Importing mkl
Importing breads


In [2]:
%matplotlib widget

In [3]:
print("Reading planet file")
planet_btsettl = "/scr3/jruffio/models/BT-Settl/BT-Settl_M-0.0_a+0.0/lte018-5.0-0.0a+0.0.BT-Settl.spec.7"
arr = np.genfromtxt(planet_btsettl, delimiter=[12, 14], dtype=np.float64,
                converters={1: lambda x: float(x.decode("utf-8").replace('D', 'e'))})
model_wvs = arr[:, 0] / 1e4
model_spec = 10 ** (arr[:, 1] - 8)

tr_files = os.listdir(tr_dir)
if "plots" in tr_files:
    tr_files.remove("plots")
tr_counter = 0
tr_total = len(tr_files)

Reading planet file


In [4]:
filename = files[1]
rvs = np.array([0])
ys = np.arange(0, 2)
xs = np.arange(0, 2)
flux = np.zeros((len(ys), len(xs))) * np.nan
noise = np.zeros((len(ys), len(xs))) * np.nan
print(filename)
dataobj = OSIRIS(dir_name+filename) 
nz,ny,nx = dataobj.data.shape

print("sky calibrating")
dataobj.calibrate(sky_calib_file)

spec_file = dir_name+"spectra/"+filename[:-5]+"_spectrum.fits"
print("Reading spectrum file", spec_file)
with pyfits.open(spec_file) as hdulist:
    star_spectrum = hdulist[2].data
    mu_x = hdulist[3].data
    mu_y = hdulist[4].data
    sig_x = hdulist[5].data
    sig_y = hdulist[6].data

s210626_a018002_Kn5_020.fits
sky calibrating
Reading spectrum file /scr3/jruffio/data/osiris_survey/targets/SR3/210626/first/reduced/spectra/s210626_a018002_Kn5_020_spectrum.fits


In [6]:
print("setting planet model")
minwv,maxwv= np.nanmin(dataobj.wavelengths),np.nanmax(dataobj.wavelengths)
crop_btsettl = np.where((model_wvs > minwv - 0.2) * (model_wvs < maxwv + 0.2))
model_wvs = model_wvs[crop_btsettl]
model_spec = model_spec[crop_btsettl]
model_broadspec = dataobj.broaden(model_wvs,model_spec)
planet_f = interp1d(model_wvs, model_broadspec, bounds_error=False, fill_value=np.nan)

setting planet model


In [7]:
tr_counter = (tr_counter + 1) % tr_total
tr_file = tr_dir + tr_files[tr_counter]
print("Reading transmission file", tr_file)
with pyfits.open(tr_file) as hdulist:
    transmission = hdulist[0].data

Reading transmission file /scr3/jruffio/data/osiris_survey/targets/SR3/210626/first/reduced/spectra/s210626_a024002_Kn5_020_spectrum.fits


In [8]:
print("setting reference position")
dataobj.set_reference_position((np.nanmedian(mu_y), np.nanmedian(mu_x)))
print(dataobj.refpos)

setting reference position
(35.19478548211935, 25.014189272358163)


In [9]:
dat = deepcopy(dataobj.data)

In [ ]:
plt.figure()
plt.imshow(np.nanmedian(dataobj.data, axis=0))
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
flux_ratio = 1
location = (10, -10)
inject_planet(dataobj, location, planet_f, spec_file, transmission, flux_ratio)

planet model is interp1d
reading star info
start injection
normalizing and adding to data


In [ ]:
plt.figure()
plt.imshow(np.nanmedian(dataobj.data, axis=0))
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
dataobj.data = deepcopy(dat)

In [30]:
plt.close('all')

In [31]:
star_loc = int(np.nanmedian(mu_y)), int(np.nanmedian(mu_x))
target_loc = int(np.nanmedian(mu_y)+location[1]), int(np.nanmedian(mu_x)+location[0])
star_loc, target_loc

((35, 25), (25, 35))

In [32]:
boxw = 10
print(np.nansum(dataobj.data[:, star_loc[1]-boxw:star_loc[1]+boxw, star_loc[0]-boxw:star_loc[0]+boxw]))
print(np.nansum(dat[:, star_loc[1]-boxw:star_loc[1]+boxw, star_loc[0]-boxw:star_loc[0]+boxw]))
print(np.nansum(dataobj.data[:, target_loc[1]-boxw:target_loc[1]+boxw, target_loc[0]-boxw:target_loc[0]+boxw]))
print(np.nansum(dat[:, target_loc[1]-boxw:target_loc[1]+boxw, target_loc[0]-boxw:target_loc[0]+boxw]))

6618374.0
5479881.0
6676458.0
1646298.1


In [15]:
np.nanmedian(sig_x), np.nanmedian(sig_y)

(1.4360245838637633, 1.7309670749580277)

In [48]:
nonlin_paras = [0, location[0], location[1]] # rv (km/s), y (pix), x (pix)
fm_paras = {"planet_f":planet_f,"transmission":transmission,"star_spectrum":star_spectrum,
            "boxw":5,"nodes":5,"psfw":(1, 1),
            "badpixfraction":0.75,"optimize_nodes":True}
fm_func = hc_no_splinefm
dataobj.set_noise()
# plt.plot(dataobj.data[:, 37, 44])
# plt.show()
# exit()

# log_prob,log_prob_H0,rchi2,linparas,linparas_err = fitfm(nonlin_paras,dataobj,fm_func,fm_paras)
# print(log_prob,log_prob_H0,rchi2,linparas,linparas_err)
d, M, s = fm_func(nonlin_paras,dataobj,**fm_paras)

validpara = np.where(np.sum(M,axis=0)!=0)
M = M[:,validpara[0]]
d = d / s
M = M / s[:, None]
print(M.shape, d.shape, s.shape)
from scipy.optimize import lsq_linear
paras = lsq_linear(M, d).x
m = np.dot(M,paras)

print("plotting")

plt.figure()
plt.subplot(2,1,1)
plt.plot(d,label="data")
plt.plot(m,label="model")
plt.plot(paras[0]*M[:,0],label="planet model")
plt.plot(m-paras[0]*M[:,0],label="starlight model")
plt.legend()
plt.subplot(2,1,2)
plt.plot(M[:,0]/np.max(M[:,0]),label="planet model")
for k in range(M.shape[-1]-1):
    plt.plot(M[:,k+1]/np.nanmax(M[:,k+1]),label="starlight model {0}".format(k+1))
plt.legend()

paras_H0 = lsq_linear(M[:,1::], d).x
m_H0 = np.dot(M[:,1::] , paras_H0)
r_H0 = d  - m_H0
r = d - m

plt.figure()
plt.plot(np.cumsum((r) ** 2),label="Residuals")
plt.plot(np.cumsum((r_H0) ** 2),label="Residuals H0")
plt.plot(np.cumsum((r_H0) ** 2 - (r) ** 2),label="Residuals H0 - H1")
plt.legend()
plt.show()
print(paras)

(11625, 126) (11625,) (11625,)
plotting


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[9.32584746e-01 3.23568425e-04 2.95762012e-04 2.75367952e-04
 2.65675935e-04 2.43270989e-04 6.68022881e-04 7.03894104e-04
 7.05934239e-04 7.26486758e-04 7.43367707e-04 1.16498145e-03
 1.33440432e-03 1.36803694e-03 1.42703475e-03 1.42289080e-03
 1.11569608e-03 1.21958464e-03 1.20253023e-03 1.20601521e-03
 1.15717341e-03 5.64001743e-04 5.64406525e-04 5.21650917e-04
 5.11017003e-04 5.23158360e-04 5.38895275e-04 5.57019604e-04
 5.67270545e-04 5.85878149e-04 5.75906525e-04 2.21598898e-03
 2.45324778e-03 2.45377291e-03 2.52359911e-03 2.44096593e-03
 4.19893061e-03 4.75816502e-03 4.76062555e-03 4.92435774e-03
 4.78660871e-03 3.19033700e-03 3.59736445e-03 3.60806658e-03
 3.72025532e-03 3.63309553e-03 1.03317991e-03 1.11772988e-03
 1.12807593e-03 1.14397360e-03 1.13132403e-03 7.82573786e-04
 7.93208339e-04 8.09803396e-04 8.37597091e-04 8.38056745e-04
 3.55937997e-03 3.97726456e-03 3.98579592e-03 4.10072483e-03
 3.98110539e-03 6.95576333e-03 7.82990082e-03 7.81075913e-03
 8.04249447e-03 7.765177

In [49]:
fm_paras = {"planet_f":planet_f,"transmission":transmission,"star_spectrum":star_spectrum,
            "boxw":5,"nodes":5,"psfw":(1, 1),
            "badpixfraction":0.75,"optimize_nodes":True}
fm_func = hc_no_splinefm
dataobj.set_noise()
out = search_planet([rvs,[location[0]],[location[1]]],dataobj,fm_func,fm_paras,numthreads=numthreads)

In [50]:
N_linpara = (out.shape[-1]-2)//2
out[0, 0, 0, 3], out[0, 0, 0, 3+N_linpara]

(0.9325847463455184, 0.0027933993936540467)

In [35]:
location

(10, -10)